# System Identification Guide: RecursiveLeastSquaresAgent

This notebook provides a guide to the `RecursiveLeastSquaresAgent`. This agent is a powerful tool for **online parameter estimation**. It can observe the input and output of a system in real-time and continuously update its estimate of the system's underlying parameters.

## 1. Theoretical Background

**Recursive Least Squares (RLS)** is an adaptive filter algorithm that recursively finds the coefficients that minimize a weighted linear least squares cost function. It's "recursive" because it doesn't need to process the entire history of data at each step; instead, it just needs its previous estimate and the newest data point to update its guess.

The `RLSAgent` in the CHS SDK is specifically designed to identify the parameters of a first-order linear system described by the equation:

`y(k) = a₁ * y(k-1) + b₁ * u(k-1)`

Where `y` is the system's output (e.g., outflow), `u` is the system's input (e.g., inflow), and `a₁` and `b₁` are the unknown parameters the agent tries to estimate.

In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np

# Add the project root to the path
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

from water_system_sdk.src.chs_sdk.modules.control.rls_estimator import RecursiveLeastSquaresAgent

## 2. Code Example: Estimating a System's Parameters

We will create a "virtual" system with known `a₁` and `b₁` parameters. Then, we will generate a random input signal `u` and calculate the corresponding output `y`. We will feed this input/output data to the `RLSAgent` at each time step and watch as its parameter estimates converge to the true values.

In [ ]:
# 1. Define the "True" System and the RLS Agent
true_a1 = 0.95
true_b1 = 0.5

rls_agent = RecursiveLeastSquaresAgent(
    initial_params={"a1": 0.0, "b1": 0.0}, # Start with no knowledge
    forgetting_factor=0.99
)

# 2. Simulation Setup
n_steps = 100
u_signal = np.random.rand(n_steps) * 10 # A random input signal
y_signal = np.zeros(n_steps)
y_prev = 0
u_prev = 0

history = {
    "a1_estimate": [],
    "b1_estimate": []
}

# 3. Simulation Loop
for k in range(n_steps):
    # Simulate the "true" system to get the current output
    y_signal[k] = true_a1 * y_prev + true_b1 * u_prev
    
    # Feed the current input and output to the RLS agent
    rls_agent.step(inflow=u_signal[k], observed_outflow=y_signal[k])
    
    # Get the agent's current best guess
    current_estimates = rls_agent.get_state()
    
    # Record history
    history["a1_estimate"].append(current_estimates['a1'])
    history["b1_estimate"].append(current_estimates['b1'])
    
    # Update state for the next iteration
    y_prev = y_signal[k]
    u_prev = u_signal[k]

print("Simulation complete.")
print(f"Final a1 estimate: {history['a1_estimate'][-1]:.4f} (True value: {true_a1})")
print(f"Final b1 estimate: {history['b1_estimate'][-1]:.4f} (True value: {true_b1})")

## 4. Visualization

Plotting the estimated parameters over time shows how the RLS algorithm converges.

In [ ]:
time_axis = np.arange(n_steps)

plt.figure(figsize=(12, 6))
plt.plot(time_axis, history['a1_estimate'], label='a1 Estimate')
plt.axhline(y=true_a1, color='r', linestyle='--', label=f'True a1 = {true_a1}')
plt.plot(time_axis, history['b1_estimate'], label='b1 Estimate')
plt.axhline(y=true_b1, color='g', linestyle='--', label=f'True b1 = {true_b1}')

plt.title('RLS Parameter Estimation Over Time')
plt.xlabel('Time Step (k)')
plt.ylabel('Parameter Value')
plt.legend()
plt.grid(True)
plt.ylim(0, 1.2) # Set y-axis limits for better visualization
plt.show()

The plot shows that, starting from zero, the agent's estimates for `a₁` and `b₁` rapidly converge towards their true values. This demonstrates the power of the `RecursiveLeastSquaresAgent` to learn the dynamics of a system from its input-output data.